# Jacobi to Cartesian Coordinates

In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
from jacobi import make_data_jacobi, make_dataset_cart_to_jac, make_dataset_jac_to_cart
from jacobi import CartesianToJacobi, JacobiToCartesian
from jacobi import make_model_cart_to_jac, make_model_jac_to_cart

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
N = 100
num_body = 3
data = make_data_jacobi(N=N, num_body=num_body)
m = data['m']
q = data['q']
qj = data['qj']
vj = data['vj']

**Review one example A Matrix as a Sanity Check**

In [6]:
# Cumulative mass
M = np.cumsum(m, axis=-1)
M_tot = keras.layers.Reshape(target_shape=(1,))(M[:, num_body-1])

# Assemble num_body x num_body square matrix converting from q to r
A_shape = (N, num_body, num_body)
A_ = np.zeros(A_shape)
A_[:, 0, :] = m / M_tot
for i in range(1, num_body):
    A_[:, i, 0:i] = -m[:, 0:i] / M[:, i-1:i]
    A_[:, i, i] = 1.0

In [7]:
A_[0]

array([[ 0.98923701,  0.00788368,  0.00287931],
       [-1.        ,  1.        ,  0.        ],
       [-0.9920935 , -0.00790645,  1.        ]])

**Create a Dataset**

In [27]:
# Create a tensorflow Dataset instance in both directions
N = 1024
num_body = 3
batch_size = 64

ds_c2j = make_dataset_cart_to_jac(N=N, num_body=num_body, batch_size=batch_size)
ds_j2c = make_dataset_jac_to_cart(N=N, num_body=num_body, batch_size=batch_size)

In [28]:
# Example batch - cartesian to jacobi
cart, jac = list(ds_c2j.take(1))[0]

In [29]:
# Unpack tensors
m = cart['m']
q = cart['q']
v = cart['v']
qj = jac['qj']
vj = jac['vj']
mu = jac['mu']

# Review shapes
print(f'Example batch sizes:')
print(f'm  = {m.shape}')
print(f'q  = {q.shape}')
print(f'v  = {v.shape}')
print(f'qj = {qj.shape}')
print(f'vj = {vj.shape}')
print(f'mu = {mu.shape}')

Example batch sizes:
m  = (64, 3)
q  = (64, 3, 3)
v  = (64, 3, 3)
qj = (64, 3, 3)
vj = (64, 3, 3)
mu = (64, 3)


### Check the Conversion Against the Dataset Using Layers

In [11]:
def rms(x):
    return np.sqrt(np.mean(np.square(x)))

In [12]:
# Compute Jacobi coordinates from Cartesian
qj_calc, vj_calc, mu_calc = CartesianToJacobi()([m, q, v])

# Error vector
err_qj = qj_calc - qj
err_vj = vj_calc - vj

# RMS Errors
rms_qj = rms(err_qj)
rms_vj = rms(err_vj)

# Display RMS errors
print(f'RMS Error qj: {rms_qj:5.3e}')
print(f'RMS Error vj: {rms_vj:5.3e}')

RMS Error qj: 3.471e-10
RMS Error vj: 3.881e-11


In [13]:
# Compute Cartesian coordinates from Jacobi
q_calc, v_calc = JacobiToCartesian()([m, qj, vj])

# Error vector
err_q = q_calc - q
err_v = v_calc - v

# RMS Errors
rms_q = rms(err_q)
rms_v = rms(err_v)

# Display RMS errors
print(f'RMS Error q: {rms_q:5.3e}')
print(f'RMS Error v: {rms_v:5.3e}')

RMS Error q: 9.534e-07
RMS Error v: 1.075e-07


***Conversion in Both Directions is Very Accurate***

### Check the Conversion Against the Dataset Using Models

In [14]:
model_c2j = make_model_cart_to_jac(num_body=3)

In [15]:
model_c2j.summary()

Model: "cartesian_to_jacobi"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
m (InputLayer)                  [(64, 3)]            0                                            
__________________________________________________________________________________________________
q (InputLayer)                  [(64, 3, 3)]         0                                            
__________________________________________________________________________________________________
v (InputLayer)                  [(64, 3, 3)]         0                                            
__________________________________________________________________________________________________
c2j (CartesianToJacobi)         ((64, 3, 3), (64, 3, 0           m[0][0]                          
                                                                 q[0][0]        

In [18]:
# Inputs to compile the c2e model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'qj': keras.losses.MeanSquaredError(),
        'vj': keras.losses.MeanSquaredError(),
        'mu': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'qj': 1.0,
                'vj': 1.0,
                'mu': 1.0}

In [19]:
# Compile the c2e model
model_c2j.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [32]:
# Invoke model manually with numpy arrays
qj_calc, vj_calc, mu = model_c2j([m, q, v])

# Error vector
err_qj = qj_calc - qj
err_vj = vj_calc - vj

# RMS Errors
rms_qj = rms(err_qj)
rms_vj = rms(err_vj)

# Display RMS errors
print(f'RMS Error qj: {rms_qj:5.3e}')
print(f'RMS Error vj: {rms_vj:5.3e}')

RMS Error qj: 3.982e-08
RMS Error vj: 0.000e+00


In [31]:
print(f'Output shapes:')
print(f'qj: {qj.shape}')
print(f'vj: {vj.shape}')
print(f'mu: {mu.shape}')

Output shapes:
qj: (64, 3, 3)
vj: (64, 3, 3)
mu: (64, 3)


In [23]:
model_c2j.predict({'m': m, 'q': q, 'v': v})

FailedPreconditionError:  Error while reading resource variable _AnonymousVar6 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar6/N10tensorflow3VarE does not exist.
	 [[node c2j/A_row_2/concat/ReadVariableOp (defined at <ipython-input-23-96f84fef3ead>:1) ]] [Op:__inference_keras_scratch_graph_911]

Function call stack:
keras_scratch_graph
